In [34]:
import pandas as pd

col_names=['OS', 'opcode', 'execute_command', 'malware_flag'] 
datadf = pd.read_csv("../datas/readAndwrtieCSV.csv",names=col_names, header=None) 
# data.info()
# data.head()

In [58]:
datadf.shape[0]

512

In [35]:
# Import Elasticsearch package 
from elasticsearch import Elasticsearch 
# Connect to the elastic cluster
elsearch=Elasticsearch([{'host':'localhost','port':9200}])
elsearch

<Elasticsearch([{'host': 'localhost', 'port': 9200}])>

In [36]:
index_name = 'test_iotlog' # Database
documenttype_name = 'test_year' # Type of the document

In [40]:
template={
    "datetime": '',
    "machine_name": '',
    "unique_ip": '',
    "OS": '',
    "opcode": '',
    "execute_command": '',
    "malware_flag": 0, # good : 0, malware : 1
    "running_flag": 0, # machine running rate good : 0, malware : 1
}

In [65]:
import random
from datetime import datetime

machine_name_type = ['Google Home Voice Controller',
            'Amazon Echo Plus Voice Controller',
            'Amazon Dash Button',
            'August Doorbell Cam',
            'August Smart Lock',
            'Kuri Mobile Robot',
            'Belkin WeMo Smart Light Switch',
            'Footbot Air Quality Monitor']
machine_name_type_length = len(machine_name_type)-1

unique_ip = ['211.201.165.134',
            '201.101.161.31',
            '251.121.161.33',
            '221.201.164.34',
            '111.211.162.35',
            '101.201.165.36',
            '121.101.161.37',
            '141.202.162.38',]

datadf_row = datadf.shape[0] - 1

template_count = 5
templage_periods = 3

template_date = pd.date_range(end = pd.datetime.today(), periods = templage_periods).to_pydatetime().tolist()
template_date

[datetime.datetime(2019, 10, 17, 1, 51, 47, 29646),
 datetime.datetime(2019, 10, 18, 1, 51, 47, 29646),
 datetime.datetime(2019, 10, 19, 1, 51, 47, 29646)]

In [68]:
for range_date in template_date:
    documenttype_name = str(range_date.strftime('%Y-%m-%d'))
    for i in range(0,template_count):
#         template["datetime"] = str(datetime.now())
        template["datetime"] = str(range_date)

        machine_name_random = random.randrange(0, machine_name_type_length)
        template["machine_name"] = machine_name_type[machine_name_random]
        template["unique_ip"] = unique_ip[machine_name_random]

        datadf_random_number = random.randrange(0, datadf_row)
        template["OS"] = datadf['OS'].iloc[datadf_random_number]
        template["opcode"] = datadf['opcode'].iloc[datadf_random_number]
        template["execute_command"] = datadf['execute_command'].iloc[datadf_random_number]
        template["malware_flag"] = int(datadf['malware_flag'].iloc[datadf_random_number])

        template["running_flag"] = random.randrange(0, 1)

        elsearch.index(index=index_name,doc_type=documenttype_name,body=template)